In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pathlib import Path

def get_ingredients_section(section):
    
    page = requests.get(f'{section}')
    soup = BeautifulSoup(page.content, 'html.parser')
    ingredient = []
    
    for a in soup.find_all('a', class_ ="promo promo__main_course" ):
        npage = requests.get(f'https://www.bbc.co.uk{a["href"]}')
        soup = BeautifulSoup(npage.content, 'html.parser')
        ingredient = []
    
        for a in soup.find_all('li', class_ = "recipe-ingredients__list-item"):
            ingredient.append(a.get_text())
        
        ingredients.append(ingredient)
    ingredients = pd.DataFrame(ingredients)
        
        #output_file = 'cuisines_mex_ingredients.csv'
        #output_dir = Path('/Users/georgesdelrieu/Desktop/Final_project')
        
    return ingredient

In [2]:
def get_ingredients_url(url):
    
    page = requests.get(f'{url}')
    soup = BeautifulSoup(page.content, 'html.parser')
    ingredient = []
    
    for a in soup.find_all('li', class_ = "recipe-ingredients__list-item"):
        ingredient.append(a.get_text())

        
    return ingredient

In [5]:
cd

/Users/georgesdelrieu


In [6]:
cd /Users/georgesdelrieu/code/g-delrieu/greenr/Parser_training/ingredient-phrase-tagger

/Users/georgesdelrieu/code/g-delrieu/greenr/Parser_training/ingredient-phrase-tagger


In [130]:
## needs to be run from /Users/georgesdelrieu/code/g-delrieu/greenr/Parser_training/ingredient-phrase-tagger

import re
import subprocess
import json
import pandas as pd
import numpy as np


def url_to_df(url):

    recipe = get_ingredients_url(url)

    our_punctuation = '!"#$%&\'())*+-:;<=>?@[\\]^_`{|}~'
    
    my_list = []
    
    names = []
    units = []
    qtys = []

    for ingredient in recipe:
    
        for punctuation in our_punctuation:
        # cleaning for common issues 
            ingredient = ingredient.replace(punctuation, '')
            
        ingredient = ingredient.replace('can', '')
        ingredient = ingredient.replace('package', '')
        ingredient = ingredient.replace('container', '')
        ingredient = ingredient.replace('eggs eggs', 'eggs')
        ingredient = ingredient.replace('⅓', '.33')
        ingredient = ingredient.replace('½', '.5')
        ingredient = ingredient.replace('¼', '.25')
        ingredient = ingredient.replace('¾', '.75')
        ingredient = ingredient.replace('tsp', 'teaspoon')
        ingredient = ingredient.replace('tbsp', 'tablespoon')

        #ingredient = re.sub("^.*\(", "", ingredient)
        try:
            ingredient = re.match("^(.+?),", ingredient).group(0)
        except:
            pass
        
    #running in the model    
        my_path = '/Users/georgesdelrieu/code/g-delrieu/greenr/Parser_training/ingredient-phrase-tagger/my_ingredients'
        parsed_ingredient = subprocess.check_output(f"echo {ingredient} | parsing_tools/parse-ingredients.py --model-file parsing_tools/20200825_0846-nyt-ingredients-snapshot-2015-461547e.crfmodel", shell=True)
        parsed_ingredient = json.loads(parsed_ingredient)
        
        my_list.append(parsed_ingredient)
        
    #appending to separate lists taking into account edge cases
        if 'name' in parsed_ingredient[0].keys():

            tmp = parsed_ingredient[0]['name']
            useless_quantifiers = ['oz', 'fl', 'ounce']
            
            try:
                names.append(re.search("[^\d]*$", tmp).group(0))
            except:
                names.append(tmp)

                    
            #else: 
            #    names.append(tmp)
        else:
            names.append(np.nan)

        if 'gram' in parsed_ingredient[0]['input']:
            units.append('gram')
        elif 'milliliters' in parsed_ingredient[0]['input']:
            units.append('ml')
        elif 'unit' in parsed_ingredient[0].keys():
            units.append(parsed_ingredient[0]['unit'])
        else:
            units.append('unit')
            
            
        if 'qty' in parsed_ingredient[0].keys():
            qtys.append(parsed_ingredient[0]['qty'])
        else:
            try:
                qtys.append(float(parsed_ingredient[0]['input'][:3]))
            except:
                qtys.append(np.nan)


    final_df = pd.DataFrame(list(zip(qtys, units, names)), columns = ['qty', 'unit', 'name'])
    final_df = final_df[final_df['name'].notna()]
    final_df = final_df[final_df['unit'].notna()]
    final_df = final_df[final_df['unit'] != 'teaspoon']
        
    return final_df, my_list

In [134]:
ls

MANIFEST.in               greenr/                   raw_data/
Makefile                  greenr.egg-info/          requirements.txt
README.md                 ingredient_phrase_tagger/ scripts/
build/                    notebooks/                setup.py
dist/                     parsing_tools/            tests/


In [135]:
subprocess.check_output(f"echo 2 cups water | parsing_tools/parse-ingredients.py --model-file parsing_tools/20200825_0846-nyt-ingredients-snapshot-2015-461547e.crfmodel", shell=True)


b'[\n  {\n    "display": "<span class=\'qty\'>2</span><span class=\'unit\'>cups</span><span class=\'name\'>water</span>", \n    "input": "2 cups water", \n    "name": "water", \n    "qty": "2", \n    "unit": "cup"\n  }\n]\n'

In [136]:
%%time
final_df, my_list = url_to_df('https://www.bbc.co.uk/food/recipes/traditional_harissa_72623')

CPU times: user 829 ms, sys: 46.5 ms, total: 876 ms
Wall time: 1.84 s


In [137]:
final_df

,qty,unit,name
0,500,gram,oz red peppers
1,NaN,unit,red chillies
2,6,unit,garlic cloves
3,4,tablespoon,olive oil


In [109]:
my_list

[[{'display': "<span class='name'>500 grams/1lb 2 oz red peppers</span><span class='other'>,</span>",
   'input': '500 grams/1lb 2 oz red peppers ,',
   'name': '500 grams/1lb 2 oz red peppers',
   'other': ','}],
 [{'display': "<span class='name'>1–4 red chillies</span><span class='other'>,</span>",
   'input': '1–4 red chillies ,',
   'name': '1–4 red chillies',
   'other': ','}],
 [{'display': "<span class='qty'>6</span><span class='other'>large</span><span class='name'>garlic cloves</span><span class='other'>,</span>",
   'input': '6 large garlic cloves ,',
   'name': 'garlic cloves',
   'other': 'large ,',
   'qty': '6'}],
 [{'display': "<span class='qty'>4</span><span class='unit'>tablespoon</span><span class='name'>olive oil</span><span class='other'>,</span>",
   'input': '4 tablespoon olive oil ,',
   'name': 'olive oil',
   'other': ',',
   'qty': '4',
   'unit': 'tablespoon'}],
 [{'comment': 'ground',
   'display': "<span class='qty'>1</span><span class='unit'>teaspoon</span